In [1]:
import logging
import os

from obspy.clients.fdsn import Client
import numpy as np
import obspy
import matplotlib.pyplot as plt
import datetime
from datetime import timedelta
import pandas as pd
import dask
from dask.diagnostics import ProgressBar

from obspy.clients.fdsn.client import Client
from obspy.core.utcdatetime import UTCDateTime
from obspy import Stream

from pnwstore.mseed import WaveformClient
import torch
import numpy as np
from tqdm import tqdm
import time 
import pandas as pd
import gc
import seisbench.models as sbm
from ELEP.elep.ensemble_statistics import ensemble_statistics
from ELEP.elep.ensemble_coherence import ensemble_semblance 
from ELEP.elep.trigger_func import picks_summary_simple
from picking_utils import run_detection, stacking

device = torch.device("cpu")

# Define clients
client_inventory = Client('IRIS')
client_waveform = WaveformClient()
client_ncedc = Client('NCEDC')

# Parameters
year1 = 2012
filepath = "../data/catalog_picks/"
os.makedirs(filepath,exist_ok=True)

twin = 6000     # length of time window
step = 3000     # step length
l_blnd, r_blnd = 500, 500

# Now create your list of days to loop over!
t1 = datetime.datetime(year=year1,month=1,day=1)
t2 = datetime.datetime(year=year1+1,month=1,day=1)
time_bins = pd.to_datetime(np.arange(t1,t2,pd.Timedelta(1,'days')))

inventory = client_inventory.get_stations(network="C8,7D,7A,CN,NV,UW,UO,NC,BK,TA,OO,PB,X6,Z5,X9", station="*", minlatitude=40,minlongitude=-127,maxlatitude=50,maxlongitude=-123, starttime=t1.strftime('%Y%m%d'),endtime=t2.strftime('%Y%m%d'))



# Make a list of networks and stations
networks_stas = []
for i in range(len(inventory)):
    network = inventory[i].code
    
    for j in range(len(inventory[i])):
        networks_stas.append([network,inventory[i].stations[j].code])

networks_stas =np.array(networks_stas)
    
# download models
pretrain_list = ["pnw","ethz","instance","scedc","stead","geofon"]
pn_pnw_model = sbm.EQTransformer.from_pretrained('pnw')
pn_ethz_model = sbm.EQTransformer.from_pretrained("ethz")
pn_instance_model = sbm.EQTransformer.from_pretrained("instance")
pn_scedc_model = sbm.EQTransformer.from_pretrained("scedc")
pn_stead_model = sbm.EQTransformer.from_pretrained("stead")
pn_geofon_model = sbm.EQTransformer.from_pretrained("geofon")

# Combine that list of days with the list of stations and networks
# We are essentially creating a list of the number of tasks we have to do with the information that is unique to each task; we will do them in parallel
task_list = []
for i in range(len(networks_stas)):
	for t in time_bins:
		task_list.append([networks_stas[i][0], networks_stas[i][1],t])
        
# Now we start setting up a parallel operation using a package called Dask.
t1 = obspy.UTCDateTime(task_list[0][2])
t2 = obspy.UTCDateTime(t1 + pd.Timedelta(1,'days'))
network = task_list[0][0]
station = task_list[0][1]
run_detection(network,station,t1,t2,filepath,twin,step,l_blnd,r_blnd)
# @dask.delayed
# def loop_days(task,filepath,twin,step,l_blnd,r_blnd):

#     # Define the parameters that are specific to each task
#     t1 = obspy.UTCDateTime(task[2])
#     t2 = obspy.UTCDateTime(t1 + pd.Timedelta(1,'days'))
#     network = task[0]
#     station = task[1]

#     #print network and station
#     # Call to the function that will perform the operation and write the results to file
#     run_detection(network,station,t1,t2,filepath,twin,step,l_blnd,r_blnd)
    
#     return


# # # Now we set up the parallel operation
# # # The below builds a framework for the computer to run in parallel. This doesn't actually execute anything.
# lazy_results = [loop_days(task,filepath,twin,step,l_blnd,r_blnd) for task in task_list]
    

# # # The below actually executes the parallel operation!
# # # It's nice to do it with the ProgressBar so you can see how long things are taking.
# # # Each operation should also write a file so that is another way to check on progress.
# # with ProgressBar():
# #     #################################
# #     # Add scheduler = 'single-threaded'
# # 	dask.compute(lazy_results, scheduler='single-threaded') 
    


/home/hbito/cascadia_obs_ensemble/workflow_2012/picking_utils.py:166: RuntimeWarning: invalid value encountered in divide
  windows_max[iseg, :] = windows[iseg, :] / (np.max(np.abs(windows[iseg, :]), axis=-1, keepdims=True))


All prediction shape: (2, 5, 3599, 6000)


NameError: name 'twin' is not defined

In [3]:
!pip3 freeze > ../requirements.txt

In [2]:
!pip3 


Usage:   
  pip <command> [options]

Commands:
  install                     Install packages.
  download                    Download packages.
  uninstall                   Uninstall packages.
  freeze                      Output installed packages in requirements format.
  list                        List installed packages.
  show                        Show information about installed packages.
  check                       Verify installed packages have compatible dependencies.
  search                      Search PyPI for packages.
  wheel                       Build wheels from your requirements.
  hash                        Compute hashes of package archives.
  completion                  A helper command used for command completion.
  help                        Show help for commands.

General Options:
  -h, --help                  Show help.
  --isolated                  Run pip in an isolated mode, ignoring
                              environment variables and user conf

In [ ]:
: